In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import repeat

In [16]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [17]:
soup = BeautifulSoup(website_url,'lxml')

In [18]:
# get the table
my_table = soup.find("table", { "class" : "wikitable sortable"})

In [19]:
# convert BeautifulSoup tags to string list
def convert_to_list(bs4row):
    list_bs4row = bs4row.findAll(["td","th"])
    return [bs4.get_text().strip() for bs4 in list_bs4row]

In [20]:
# get the table 
rows=my_table.findAll("tr")

# first row is header
header = convert_to_list(rows[0])
header

['Postal code', 'Borough', 'Neighborhood']

In [21]:
# convert to list of list
my_data = [convert_to_list(r) for r in rows[1:]]

In [22]:
df = pd.DataFrame(my_data,columns=header)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [23]:
df = df[~df['Borough'].isin(['Not assigned'])]
df.head(10)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [27]:
df['Neighborhood'] = df.apply(lambda row: row['Borough'] if row['Neighborhood']=='Not assigned' else row['Neighborhood'], axis=1)
df.head(10)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [28]:
df = df.groupby(['Postal code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
df.shape

(103, 3)

In [32]:
fn_coor = "Geospatial_Coordinates.csv"
df_coor = pd.read_csv(fn_coor)
df_coor.head()


FileNotFoundError: [Errno 2] File b'Geospatial_Coordinates.csv' does not exist: b'Geospatial_Coordinates.csv'

In [33]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0356884f076147cc9076039be04f0609 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XZo-whP88QfW7XM327US_pRW3UXusS2HTZHVih8PfXf6',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_0356884f076147cc9076039be04f0609.get_object(Bucket='courseracapstone-donotdelete-pr-ei44vha4v5uofu',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_coor = pd.read_csv(body)
df_coor.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
